In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [31]:
path = '/home/Kaggle-challenge/workspace/data/'


train_d = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submissions.csv')

train_raw = train_d.copy()

col = ['severe_toxicity', 'obscene',
       'threat', 'insult', 'identity_attack', 'sexual_explicit']

X = train_d.drop(['toxicity'], axis=1)
X = X[col]
y = train_d[['toxicity']]

y = (y['toxicity'] >= 0.5).astype(int)

In [32]:
# scaler = StandardScaler()
scaler = MinMaxScaler()
scaler.fit(X.loc[:,col])

MinMaxScaler()

In [39]:
X.loc[:,col] = scaler.transform(X.loc[:,col])

In [38]:
test = pd.read_csv('/home/Kaggle-challenge/subtype.csv')
test = test[col]
# test.loc[:,col] = scaler.transform(test.loc[:,col])
test

,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,5.040608e-08,0.000121,0.000065,0.000930,0.000201,0.000055
1,3.800427e-07,0.000326,0.000422,0.018133,0.004189,0.000263
2,1.861475e-07,0.000097,0.000121,0.000355,0.000119,0.000042
3,2.383215e-06,0.001420,0.000720,0.025394,0.003655,0.019518
4,4.359460e-07,0.001329,0.000442,0.047236,0.002245,0.000518
...,...,...,...,...,...,...
431410,5.737811e-06,0.000327,0.003947,0.012833,0.130369,0.000644
431411,8.808725e-08,0.000074,0.000049,0.000426,0.000123,0.000038
431412,1.264490e-07,0.000100,0.000076,0.000308,0.000139,0.000041
431413,2.205629e-07,0.000152,0.000100,0.001256,0.011294,0.000098


In [35]:
import lightgbm as lgb

lgb_clf = lgb.LGBMClassifier(num_leaves=31, objective='binary')
lgb_clf.fit(X,y)
y_pred = lgb_clf.predict(test)

[LightGBM] [Info] Number of positive: 113658, number of negative: 954927
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1477
[LightGBM] [Info] Number of data points in the train set: 1068585, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.106363 -> initscore=-2.128441
[LightGBM] [Info] Start training from score -2.128441


In [51]:
param = {'num_leaves':75,'min_data_in_leaf':100000, 'objective':'binary', 'metric':'auc'}

train_data = lgb.Dataset(X, label=y.values)
num_round= 10

bst = lgb.train(param, train_data, num_round)

y_pred_sub = bst.predict(test)

[LightGBM] [Info] Number of positive: 113658, number of negative: 954927
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029230 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 756
[LightGBM] [Info] Number of data points in the train set: 1068585, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.106363 -> initscore=-2.128441
[LightGBM] [Info] Start training from score -2.128441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [52]:
y_pred_sub

array([0.06245846, 0.0732996 , 0.06245846, ..., 0.06245846, 0.06245846,
       0.06245846])

In [53]:
submission['prediction'] = y_pred_sub

In [54]:
submission

,id,prediction
0,0,0.062458
1,1,0.073300
2,2,0.062458
3,3,0.073300
4,4,0.073300
...,...,...
431410,431410,0.073300
431411,431411,0.062458
431412,431412,0.062458
431413,431413,0.062458


In [55]:
submission.to_csv("test_l22.csv", index=False)

In [11]:
check = pd.read_csv("test_l2.csv")
check

,id,prediction
0,0,0.039379
1,1,0.039379
2,2,0.039379
3,3,0.039379
4,4,0.039379
...,...,...
431410,431410,0.039379
431411,431411,0.039379
431412,431412,0.039379
431413,431413,0.039379
